# 1. API Data Structuring

In this notebook I'll share with you the initial steps of importing MTG card data from an API; followed by the discovery process, its structuring, cleaning and export; so that it can be used on other data projects later on.

## 1.1. Resources and Setup

To start, let's set up our project imports. We'll need some components from the scipy stack, some general utility packages and a custom module I wrote to isolate the process of requesting the API.


In [1]:
# Project imports
import os   # To access environment variables
import sys  # To allow access to my local modules from this notebook

from pathlib import Path    # To ease working with filesystem paths
from datetime import date   # to manage product versions based on date

# Scipy Stack 
import numpy as np
import pandas as pd

# Setting up custom modules
project_folder = Path.cwd().parent

if str(project_folder) not in sys.path:
    sys.path.append(str(project_folder))

from modules.api_client import get_set_data

Next, the project constants should be defined. this will include general details for the project like a project identifier, the current date, and the path where any resulting file should be stored on. In this case we'll store two files: the raw data as obtained from the api to avoid repeated queries (1), and the resulting data after being cleaned (2) so that it can be used in other projects.

In [2]:
# Project Constants
PROJECT_CODE = 'mtg_demo' # A short identifier for this project.
PROJECT_DATE = str(date.today())

# Indentifiers for the files we intend on producing in this notebook
RAW_PRODUCT = 'scryfall_raw'
PROCESSED_PRODUCT= 'processed_data'

RAW_DATA_OUT_DIR = Path(os.environ['RAW_DATA_DIR_PATH'])
RAW_DATA_PATH = RAW_DATA_OUT_DIR / ('_'.join([PROJECT_CODE,
                                             RAW_PRODUCT,
                                             PROJECT_DATE]))

PROCESSED_DATA_OUT_DIR = Path(os.environ['PROCESSED_DATA_DIR_PATH'])
PROCESSED_DATA_PATH = PROCESSED_DATA_OUT_DIR / ('_'.join([PROJECT_CODE,
                                                         PROCESSED_PRODUCT,
                                                         PROJECT_DATE]))

## 1.2. Obtaining the Card Data

There are multiple choices as to where we can import data from. I chose [Scryfall's API](https://scryfall.com/docs/api) for the sake of familiarity with their platform and documentation. As i mentioned earlier, the specifics of the requesting process are kept in the api_client.py module in the modules folder of this project. 

In [3]:
raw_set_data = get_set_data(
    RAW_DATA_PATH.with_suffix('.json'),
    query='(s:mom or s:stx) -is:rebalanced'
)

2023-05-09 11:46:00.768 | INFO     | modules.api_client:get_set_data:10 - 
 - previously stored data found. skipping request.



## 1.3. Processing the Data

To first familiarize ourselves with the data we run the **`pandas.DataFrame.info()`** method. this should display the size of the dataframe; and the name, dtype, and prevalence of null values in every column. adding **`verbose=True`** as a parameter might be needed to display all the details.



In [4]:
raw_set_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571 entries, 0 to 570
Data columns (total 72 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   object             571 non-null    object        
 1   id                 571 non-null    object        
 2   oracle_id          571 non-null    object        
 3   multiverse_ids     571 non-null    object        
 4   mtgo_id            571 non-null    int64         
 5   arena_id           571 non-null    int64         
 6   tcgplayer_id       571 non-null    int64         
 7   cardmarket_id      556 non-null    float64       
 8   name               571 non-null    object        
 9   lang               571 non-null    object        
 10  released_at        571 non-null    datetime64[ns]
 11  uri                571 non-null    object        
 12  scryfall_uri       571 non-null    object        
 13  layout             571 non-null    object        
 14  highres_im

The card data we are requesting belongs to the sets March of the Machine (MOM) and Strixhaven (STX). In my experience, exploring the contents of a set with data tools accelerates the process of becoming deeply familiar with the themes and composition of the set. 

Say, we are now interested in the columns that describe gameplay features of the cards in the set to ease the process of playing with new cards. We can start by filtering the data with a list of the features relevant to gameplay. In this case we can verify what features are relevant to gameplay by reviewing the descriptions in the [Scryfall's API Documentation](https://scryfall.com/docs/api/cards).

In [5]:
gameplay_features = ['id', 'name', 'layout', 'mana_cost', 'cmc', 
                     'type_line','oracle_text','colors', 'card_faces',
                     'rarity', 'power', 'toughness', 'loyalty', 
                     'image_uris']

set_data = raw_set_data[gameplay_features]
set_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571 entries, 0 to 570
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           571 non-null    object 
 1   name         571 non-null    object 
 2   layout       571 non-null    object 
 3   mana_cost    494 non-null    object 
 4   cmc          571 non-null    int64  
 5   type_line    571 non-null    object 
 6   oracle_text  494 non-null    object 
 7   colors       494 non-null    object 
 8   card_faces   77 non-null     object 
 9   rarity       571 non-null    object 
 10  power        250 non-null    float64
 11  toughness    250 non-null    float64
 12  loyalty      5 non-null      float64
 13  image_uris   494 non-null    object 
dtypes: float64(3), int64(1), object(10)
memory usage: 66.9+ KB


Also, in the documetation we can find the following key details about the data 

1. Any row corresponding to a double-face card (dfc) has an array of json data with the features of each face in the `card_faces` column.

2. We can tell if a card is a dfc by looking at the `layout` column.

3. `cmc, colors, image_uris, layout, loyalty, mana_cost, name, power, toughness,` and `type_line` are the properties of each card face in the card_faces array if any. I'll also include the recently released `defense` feature since it's not been included in the documentation yet.


Let's verify the contents of the layout column.

In [6]:
set_data.layout.value_counts()

normal       494
transform     61
modal_dfc     16
Name: layout, dtype: int64

### 1.3.1. Extracting the properties of each card face
The `card_faces` column of the 77 non-normal layout  cards should help fill the null values of the columns with 494 non-null values. We need to write a function that extracts the features from the card face array but before that lets also label each item of the `card_faces` to distinguish the front face from the back face in a `face_type` feature.

In [7]:
def set_face_order(row):
    
    face_map = {
        0: 'front_face',
        1: 'back_face'}
    
    row['face_type'] = []
    
    if not (row['layout'] == 'transform' or row['layout'] == 'modal_dfc'):
        row['card_faces'] = [None]
        row['face_type'].append('front_face')
        return row
    
    for i, face in enumerate(row['card_faces']):
        row['face_type'].append(face_map[i])
    
    return row

In [8]:
def extract_card_face_features(row , features):
    
    if row['layout'] == 'transform' or row['layout'] == 'modal_dfc':
        for feature in features:
            try:
                row[feature] = row['card_faces'][feature]
            except KeyError:
                row[feature] = np.nan
    return row

the following set of funtions should deliver the desired data.

In [9]:
card_face_features = ['cmc', 'image_uris','layout','loyalty','mana_cost',
                      'colors','power','name','toughness','type_line',
                      'defense', 'oracle_text']

dfc_set_data = (set_data
    .apply(
        set_face_order,
        axis='columns')
    .explode(['card_faces','face_type'])
    .apply(extract_card_face_features,
           axis='columns',
           features= card_face_features)
    .drop(columns='card_faces'))



In [10]:
dfc_set_data.loc[5:8, ['name', 'face_type']]

,name,face_type
5,Aetherblade Agent,front_face
5,Gitaxian Mindstinger,back_face
6,Aether Helix,front_face
7,Ageless Guardian,front_face
8,Akki Scrapchomper,front_face


It looks mostly fine, but there are some columns with null values. let's look deeper.

In [11]:
dfc_set_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 570
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          494 non-null    float64
 1   colors       648 non-null    object 
 2   defense      36 non-null     object 
 3   face_type    648 non-null    object 
 4   id           648 non-null    object 
 5   image_uris   648 non-null    object 
 6   layout       494 non-null    object 
 7   loyalty      9 non-null      object 
 8   mana_cost    648 non-null    object 
 9   name         648 non-null    object 
 10  oracle_text  648 non-null    object 
 11  power        338 non-null    object 
 12  rarity       648 non-null    object 
 13  toughness    338 non-null    object 
 14  type_line    648 non-null    object 
dtypes: float64(1), object(14)
memory usage: 97.2+ KB


indeed the layout and cmc columns seem to be always null. we should remove them from the card_face_features list

In [12]:
card_face_features = ['image_uris', 'loyalty','mana_cost',
                      'colors','power','name','toughness','type_line',
                      'defense', 'oracle_text']

dfc_set_data = (set_data
    .apply(
        set_face_order,
        axis='columns')
    .explode(['card_faces','face_type'])
    .apply(extract_card_face_features,
           axis='columns',
           features= card_face_features)
    .drop(columns='card_faces'))

dfc_set_data.head(4)

,cmc,colors,defense,face_type,id,image_uris,layout,loyalty,mana_cost,name,oracle_text,power,rarity,toughness,type_line
0,1,[R],NaN,front_face,4620cc3b-e401-4096-b310-fed080806344,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{R},Academic Dispute,Target creature blocks this turn if able. You ...,NaN,uncommon,NaN,Instant
1,2,[W],NaN,front_face,05521edf-f47f-4e7a-aec5-cdc4ae7368c2,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{1}{W},Academic Probation,Choose one —\n• Choose a nonland card name. Op...,NaN,rare,NaN,Sorcery — Lesson
2,0,[],NaN,front_face,edf8eb51-9643-4c54-b38e-e7abea92bbe1,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,,Access Tunnel,"{T}: Add {C}.\n{3}, {T}: Target creature with ...",NaN,uncommon,NaN,Land
3,4,[G],NaN,front_face,0d7b7830-b65e-4c53-98e8-152026764e4b,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{3}{G},Accomplished Alchemist,{T}: Add one mana of any color.\n{T}: Add X ma...,2.0,rare,5.0,Creature — Elf Druid


In [13]:
dfc_set_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 570
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cmc          648 non-null    int64 
 1   colors       648 non-null    object
 2   defense      36 non-null     object
 3   face_type    648 non-null    object
 4   id           648 non-null    object
 5   image_uris   648 non-null    object
 6   layout       648 non-null    object
 7   loyalty      9 non-null      object
 8   mana_cost    648 non-null    object
 9   name         648 non-null    object
 10  oracle_text  648 non-null    object
 11  power        338 non-null    object
 12  rarity       648 non-null    object
 13  toughness    338 non-null    object
 14  type_line    648 non-null    object
dtypes: int64(1), object(14)
memory usage: 81.0+ KB


Although we don't have any more null values in the layout and cmc columns, there are still issues with the cmc column. In case you are not familiar with mtg, cmc standands for converted mana cost and it represents the numeric magnitude of the mana cost. Our problem comes up with modal dfcs, which unlike cards with transform layouts, won't necessarily share their cmc between faces. If the two faces of a modal dfc have different `mana_cost` their cmc is also likely to be different but in our data it's always the same. To resolve this issue I wrote a script (in the modules folder) to parse the mana cost and calculate the cmc. We'll apply this script to every row containing a card with modal_dfc layout.

In [14]:
from modules.mana_cost_parser import get_cmc

def set_mdfc_cmc(row):
    if row['layout'] == 'modal_dfc':
        row['cmc'] = get_cmc(row['mana_cost'])
    
    return row

In [15]:

(dfc_set_data
    .apply(set_mdfc_cmc, axis='columns')).head(4)


,cmc,colors,defense,face_type,id,image_uris,layout,loyalty,mana_cost,name,oracle_text,power,rarity,toughness,type_line
0,1,[R],NaN,front_face,4620cc3b-e401-4096-b310-fed080806344,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{R},Academic Dispute,Target creature blocks this turn if able. You ...,NaN,uncommon,NaN,Instant
1,2,[W],NaN,front_face,05521edf-f47f-4e7a-aec5-cdc4ae7368c2,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{1}{W},Academic Probation,Choose one —\n• Choose a nonland card name. Op...,NaN,rare,NaN,Sorcery — Lesson
2,0,[],NaN,front_face,edf8eb51-9643-4c54-b38e-e7abea92bbe1,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,,Access Tunnel,"{T}: Add {C}.\n{3}, {T}: Target creature with ...",NaN,uncommon,NaN,Land
3,4,[G],NaN,front_face,0d7b7830-b65e-4c53-98e8-152026764e4b,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{3}{G},Accomplished Alchemist,{T}: Add one mana of any color.\n{T}: Add X ma...,2.0,rare,5.0,Creature — Elf Druid


Now that we know everything is working correctly lets apply to the main dataset. 

In [16]:
set_data = (
    set_data
       .apply(
           set_face_order,
           axis='columns')
       .explode(['card_faces','face_type'])
       .apply(extract_card_face_features,
           axis='columns',
           features=card_face_features)
       .apply(set_mdfc_cmc,
           axis='columns')
       .drop(columns='card_faces')
       .reset_index(drop=True))

In [17]:
set_data.head(3)

,cmc,colors,defense,face_type,id,image_uris,layout,loyalty,mana_cost,name,oracle_text,power,rarity,toughness,type_line
0,1,[R],NaN,front_face,4620cc3b-e401-4096-b310-fed080806344,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{R},Academic Dispute,Target creature blocks this turn if able. You ...,NaN,uncommon,NaN,Instant
1,2,[W],NaN,front_face,05521edf-f47f-4e7a-aec5-cdc4ae7368c2,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,{1}{W},Academic Probation,Choose one —\n• Choose a nonland card name. Op...,NaN,rare,NaN,Sorcery — Lesson
2,0,[],NaN,front_face,edf8eb51-9643-4c54-b38e-e7abea92bbe1,{'small': 'https://cards.scryfall.io/small/fro...,normal,NaN,,Access Tunnel,"{T}: Add {C}.\n{3}, {T}: Target creature with ...",NaN,uncommon,NaN,Land


In [18]:
set_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cmc          648 non-null    int64 
 1   colors       648 non-null    object
 2   defense      36 non-null     object
 3   face_type    648 non-null    object
 4   id           648 non-null    object
 5   image_uris   648 non-null    object
 6   layout       648 non-null    object
 7   loyalty      9 non-null      object
 8   mana_cost    648 non-null    object
 9   name         648 non-null    object
 10  oracle_text  648 non-null    object
 11  power        338 non-null    object
 12  rarity       648 non-null    object
 13  toughness    338 non-null    object
 14  type_line    648 non-null    object
dtypes: int64(1), object(14)
memory usage: 76.1+ KB


### 1.3.2. Extracting the image uris

Since we are keeping the `image_uris` data as as simple reference to the card, we don't really need to store all the image versions for every card. We can write a function to do this for every row of the dataset.

In [19]:
# Defining a functio to extract normal image uris
def get_normal_uri(row):
    row['normal_image_uri'] = row['image_uris']['normal']
    return row

In [20]:
set_data = (
    set_data
        .apply(
            get_normal_uri,
            axis='columns')
        .drop(
            columns='image_uris'))

set_data.normal_image_uri.head(3)

0    https://cards.scryfall.io/normal/front/4/6/462...
1    https://cards.scryfall.io/normal/front/0/5/055...
2    https://cards.scryfall.io/normal/front/e/d/edf...
Name: normal_image_uri, dtype: object

### 1.3.3. Encoding the type line and color categories

The `type_line` feature describes supertypes, types and subtypes of cards through a string of text. We'd like to break it down into disctinct features to ease insight extraction. To achieve this will turn the type line into a list and encode any relevant types found in it. However, we dont really care about every single type that can be listed in the type line. We'll define a list with the types we care about otherwise our DataFrame would grow to more than a hundred columns.

In [21]:
import re

def parse_type_line(row, type_map):
    
    type_line = row['type_line'].split(' ')
    
    for segment in type_line:
        
        if re.match(r'[^a-zA-Z]', segment):
            continue
        
        if segment in type_map.keys():
            row[type_map[segment]] = 1
    
    return row

In [22]:
relevant_types=['Legendary','Instant','Creature',
                        'Battle','Enchantment','Artifact','Land','Sorcery',
                        'Planeswalker']

type_map = { r_type: 'is_'+r_type.lower() for r_type in relevant_types}

set_data = (
    set_data
        .apply(
            parse_type_line,
            type_map=type_map,
            axis='columns'))

for col in type_map.values():
    try:
        set_data[col] = (~ set_data[col].isna())
    except KeyError:
        continue

In [23]:
set_data.loc[:4, ['name', 'type_line']+ list(type_map.values())]

,name,type_line,is_legendary,is_instant,is_creature,is_battle,is_enchantment,is_artifact,is_land,is_sorcery,is_planeswalker
0,Academic Dispute,Instant,False,True,False,False,False,False,False,False,False
1,Academic Probation,Sorcery — Lesson,False,False,False,False,False,False,False,True,False
2,Access Tunnel,Land,False,False,False,False,False,False,True,False,False
3,Accomplished Alchemist,Creature — Elf Druid,False,False,True,False,False,False,False,False,False
4,Aerial Boost,Instant,False,True,False,False,False,False,False,False,False


A similar process can be done to separate the color features.

In [24]:
def parse_colors(row, color_map):
    
    #Check the color column
    colors = row['colors']
    
    row['n_colors'] = len(colors)
    
    if not colors:
        row['colors'] = 'colorless'
        row['is_colorless'] = 1
        return row
    
    row['colors'] = ''.join(colors)
    
    for color in colors:
        row[color_map[color]] = 1
    
    return row

In [25]:
color_map = {
        'W': 'is_white',
        'U': 'is_blue',
        'B': 'is_black',
        'R': 'is_red',
        'G': 'is_green',
        'colorless': 'is_colorless'
    }
set_data = (
    set_data
        .apply(
            parse_colors,
            color_map=color_map,
            axis='columns'))

for col in color_map.values():
    try:
        set_data[col] = (~ set_data[col].isna())
    except KeyError:
        continue

In [26]:
set_data.loc[45:49, ['name', 'colors', 'n_colors']+ list(color_map.values())]

,name,colors,n_colors,is_white,is_blue,is_black,is_red,is_green,is_colorless
45,Search for Blex,B,1,False,False,True,False,False,False
46,Blighted Burgeoning,G,1,False,False,False,False,True,False
47,Blightreaper Thallid,B,1,False,False,True,False,False,False
48,Blightsower Thallid,BG,2,False,False,True,False,True,False
49,Bloated Processor,B,1,False,False,True,False,False,False


In [27]:
set_data.head(7)

,cmc,colors,defense,face_type,id,is_artifact,is_battle,is_black,is_blue,is_colorless,...,loyalty,mana_cost,n_colors,name,normal_image_uri,oracle_text,power,rarity,toughness,type_line
0,1,R,NaN,front_face,4620cc3b-e401-4096-b310-fed080806344,False,False,False,False,False,...,NaN,{R},1,Academic Dispute,https://cards.scryfall.io/normal/front/4/6/462...,Target creature blocks this turn if able. You ...,NaN,uncommon,NaN,Instant
1,2,W,NaN,front_face,05521edf-f47f-4e7a-aec5-cdc4ae7368c2,False,False,False,False,False,...,NaN,{1}{W},1,Academic Probation,https://cards.scryfall.io/normal/front/0/5/055...,Choose one —\n• Choose a nonland card name. Op...,NaN,rare,NaN,Sorcery — Lesson
2,0,colorless,NaN,front_face,edf8eb51-9643-4c54-b38e-e7abea92bbe1,False,False,False,False,True,...,NaN,,0,Access Tunnel,https://cards.scryfall.io/normal/front/e/d/edf...,"{T}: Add {C}.\n{3}, {T}: Target creature with ...",NaN,uncommon,NaN,Land
3,4,G,NaN,front_face,0d7b7830-b65e-4c53-98e8-152026764e4b,False,False,False,False,False,...,NaN,{3}{G},1,Accomplished Alchemist,https://cards.scryfall.io/normal/front/0/d/0d7...,{T}: Add one mana of any color.\n{T}: Add X ma...,2.0,rare,5.0,Creature — Elf Druid
4,2,W,NaN,front_face,f7017afb-4c7c-4c8d-9c9d-3f056a55561e,False,False,False,False,False,...,NaN,{1}{W},1,Aerial Boost,https://cards.scryfall.io/normal/front/f/7/f70...,Convoke (Your creatures can help cast this spe...,NaN,common,NaN,Instant
5,2,B,NaN,front_face,dad34ae5-56b4-4394-be02-e043dc1cc23d,False,False,True,False,False,...,NaN,{1}{B},1,Aetherblade Agent,https://cards.scryfall.io/normal/front/d/a/dad...,Deathtouch\n{4}{U/P}: Transform Aetherblade Ag...,1,common,1,Creature — Human Rogue
6,2,BU,NaN,back_face,dad34ae5-56b4-4394-be02-e043dc1cc23d,False,False,True,True,False,...,NaN,,2,Gitaxian Mindstinger,https://cards.scryfall.io/normal/back/d/a/dad3...,Deathtouch\nWhenever Gitaxian Mindstinger deal...,3,common,3,Creature — Phyrexian Rogue


In [28]:
set_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cmc               648 non-null    int64 
 1   colors            648 non-null    object
 2   defense           36 non-null     object
 3   face_type         648 non-null    object
 4   id                648 non-null    object
 5   is_artifact       648 non-null    bool  
 6   is_battle         648 non-null    bool  
 7   is_black          648 non-null    bool  
 8   is_blue           648 non-null    bool  
 9   is_colorless      648 non-null    bool  
 10  is_creature       648 non-null    bool  
 11  is_enchantment    648 non-null    bool  
 12  is_green          648 non-null    bool  
 13  is_instant        648 non-null    bool  
 14  is_land           648 non-null    bool  
 15  is_legendary      648 non-null    bool  
 16  is_planeswalker   648 non-null    bool  
 17  is_red          

### 1.4. Validating the Data

With all the data extracted and formatted we can proceed to validate its integrity The first validations required come from the columns with null values. Lets verify that all null values are justified, there isn't any missing data, and that every value we do have is reasonable.

#### 1.4.1. Validating the defense column

We can begin by looking at the defense column. This column is meant to describe the number of defense counter on cards of the battle type. 


In [29]:
set_data.defense

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
643    NaN
644    NaN
645    NaN
646    NaN
647    NaN
Name: defense, Length: 648, dtype: object

In [30]:
set_data.defense.value_counts()

4    18
5    11
6     3
7     2
3     2
Name: defense, dtype: int64

the defense column only contains numeric values but it is represented as an object. Ideally we would cast it as an 8 bit integer column but since it has null values we will cast it a 16bit float instead.

In [31]:
set_data.defense = set_data.defense.astype('float16')

In [32]:
set_data.defense

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
643   NaN
644   NaN
645   NaN
646   NaN
647   NaN
Name: defense, Length: 648, dtype: float16

lets check how are the defense observations grouped between battle and non-battle cards.

In [33]:
(set_data
    .loc[:,['name','is_battle','defense']]
    .groupby('is_battle').count())

,name,defense
is_battle,,
False,612,0
True,36,36


The distribution appears to be right, so we are ready to move onto the loyalty column.

#### 1.4.2. Validating the loyalty column

The same scenario repeats for the loyalty column. This feature describe the number of loyalty counter for cards with the Planeswalker type. we can repeat the same exercise.

In [34]:
set_data.loyalty

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
643    NaN
644    NaN
645    NaN
646    NaN
647    NaN
Name: loyalty, Length: 648, dtype: object

In [35]:
set_data.loyalty.value_counts()

4.0    2
5.0    2
4      2
2.0    1
5      1
2      1
Name: loyalty, dtype: int64

Aside from being an object type feature, like in the previous case, there seems to be a discrepance on the way the loyalty value was stored for some cards. 

In [36]:
set_data.loyalty.astype('float16').value_counts()

4.0    4
5.0    3
2.0    2
Name: loyalty, dtype: int64

this is fixed when we cast them under the same dtype.

In [37]:
set_data.loyalty = set_data.loyalty.astype('float16')

Then we can continue to check the distribution under the `is_planeswalker` groups.

In [38]:
(set_data
    .loc[:, ['name','is_planeswalker','loyalty']]
    .groupby('is_planeswalker')
    .count())

,name,loyalty
is_planeswalker,,
False,639,0
True,9,9


The distribution checks out. Let's procedd to the power and toughness columns.

#### 1.4.3. Validating the power and toughness column

Finally, The power and toughness columns are meant to describe the value of the power and tougness creature and some artifact cards use to interact with other game mechanics. Let's examine the values in each of these columns to identify potential problems.


In [39]:
# Display an overview of the power column values
set_data.power

0      NaN
1      NaN
2      NaN
3      2.0
4      NaN
      ... 
643    3.0
644    NaN
645    3.0
646    1.0
647    4.0
Name: power, Length: 648, dtype: object

In [40]:
# Count the number of appearances of al non-null values in the power column
set_data.power.value_counts()

2.0     96
3.0     52
1.0     33
4.0     31
4       24
3       22
2       20
0.0     13
5.0     12
1        8
6.0      5
5        5
7.0      4
*        3
7        2
6        2
8.0      1
11.0     1
11       1
12.0     1
8        1
18.0     1
Name: power, dtype: int64

In [41]:

# Display an array with all the unique values in the power column
set_data.power.unique()

array([nan, 2.0, '1', '3', 1.0, 3.0, 6.0, 4.0, '4', 5.0, '2', '7', 7.0,
       0.0, 8.0, 11.0, '11', 12.0, '5', '8', '*', '6', 18.0], dtype=object)

When examining the power column we can find values stored on multiple formats like in the loyalty example. However we can also see that theres multiple entries with power value '*'. let's see what that's about.

In [42]:
# Defining a regularx expresion for numeric values
NUMBER_REGEX = r'^[0-9]+\.{0,1}[0-9]*$'

In [43]:
# Display observations with string value in the power column
set_data.loc[
    ((~ set_data.power
        .astype(str)
        .str.match(NUMBER_REGEX))
     & (~ set_data.power.isna())),
    ['name','power','type_line','oracle_text']
]

,name,power,type_line,oracle_text
275,"Aetherwing, Golden-Scale Flagship",*,Legendary Artifact — Vehicle,"Flying\nAetherwing, Golden-Scale Flagship's po..."
285,Winnowing Forces,*,Creature — Elf Warrior,Winnowing Forces's power and toughness are eac...
317,Vertex Paladin,*,Creature — Angel Knight,Flying\nVertex Paladin's power and toughness a...


When reviewing the oracle text of these cards. We can see that these spells have a variable power and toughness, justifying the '*' observations. So the value shouldn't be changed to another dtype. However, we still need to define a consistent format for the non-string values. We would also benefit from defining an additional column with only the static values of the power column to avoid losing the metric information of the power feature.

In [44]:
def parse_numeric_col(row, col_name, value_regex, as_type='float'):
    
    # Get the value of the col
    observation = row[col_name]
    
    # Skip it if its null
    if pd.isna(observation):
        return row
    
    # If it is not a number then ...
    if not re.match(
                value_regex,
                str(observation)):
        row[f'{col_name}_type'] = 'dynamic' # label it as dyanmic...
        row[f'static_{col_name}'] = 0.0     # set its static value as 0...
        return row                          # and exit
    
    
    row[f'{col_name}_type'] = 'static'              # Label it as static...
    row[f'static_{col_name}'] = float(observation)  # set its static value
                                                    # as float...
    row[col_name] = str(float(observation))         # and store its static 
                                                    # value as string too
    return row

In [45]:
# Parsing the power column
set_data = (set_data
        .apply(
            parse_numeric_col,
            axis='columns',
            col_name='power',
            value_regex=NUMBER_REGEX))

We can also save some space by casting the static_power column as a float16

In [46]:
# Casting the power column as float16
set_data.static_power = set_data.static_power.astype('float16')

In [47]:

# Displaing an overview of the data parsed from the static values of 
# the power column
set_data.loc[
    set_data.power_type == 'static',
    ['name','power','power_type','static_power']]

,name,power,power_type,static_power
3,Accomplished Alchemist,2.0,static,2.0
5,Aetherblade Agent,1.0,static,1.0
6,Gitaxian Mindstinger,3.0,static,3.0
8,Ageless Guardian,1.0,static,1.0
9,Akki Scrapchomper,1.0,static,1.0
...,...,...,...,...
642,Zephyr Winder,2.0,static,2.0
643,Zhalfirin Lancer,3.0,static,3.0
645,Zimone and Dina,3.0,static,3.0
646,"Zimone, Quandrix Prodigy",1.0,static,1.0


In [48]:
# Displaing an overview of the data parsed from the dynamic values of 
# the power column
set_data.loc[
    set_data.power_type == 'dynamic',
    ['name','power','power_type','static_power']]

,name,power,power_type,static_power
275,"Aetherwing, Golden-Scale Flagship",*,dynamic,0.0
285,Winnowing Forces,*,dynamic,0.0
317,Vertex Paladin,*,dynamic,0.0


In [49]:
# Count the number of appearances of al non-null values in the power column
set_data.power.value_counts()

2.0     116
3.0      74
4.0      55
1.0      41
5.0      17
0.0      13
6.0       7
7.0       6
*         3
8.0       2
11.0      2
12.0      1
18.0      1
Name: power, dtype: int64

In [50]:
# Display an array with all the unique values in the power column
set_data.power.unique()

array([nan, '2.0', '1.0', '3.0', '6.0', '4.0', '5.0', '7.0', '0.0', '8.0',
       '11.0', '12.0', '*', '18.0'], dtype=object)

We can repeat the same process for the toughness column.

In [51]:
# Display an overview of the toughness column values
set_data.toughness

0      NaN
1      NaN
2      NaN
3      5.0
4      NaN
      ... 
643    3.0
644    NaN
645    4.0
646    2.0
647    4.0
Name: toughness, Length: 648, dtype: object

In [52]:
# Count the number of appearances of al non-null values in the toughness
# column
set_data.toughness.value_counts()

2.0     78
4.0     50
3.0     48
1.0     40
3       26
4       22
5.0     20
2       12
5       11
1        8
6.0      6
6        4
0.0      3
7.0      2
*        2
8.0      1
10.0     1
7        1
11       1
12.0     1
8        1
Name: toughness, dtype: int64

In [53]:
# Display an array with all the unique values in the toughness column
set_data.toughness.unique()

array([nan, 5.0, '1', '3', 4.0, 1.0, 2.0, 6.0, '4', 3.0, '2', '5', 7.0,
       0.0, 8.0, 10.0, '7', '11', 12.0, '6', '8', '*'], dtype=object)

The toughness columns presents the same issues as the power column. It is an object type. contains a mix and match of number formats and dtypes. and some values are not numerical. We would be right to follow the same procedure.

In [54]:
# Display observations with string value in toughness column
set_data.loc[
    ((~ set_data.toughness
        .astype(str)
        .str.match(NUMBER_REGEX))
     & (~ set_data.toughness.isna())),
    ['name','toughness','type_line','oracle_text']]

,name,toughness,type_line,oracle_text
285,Winnowing Forces,*,Creature — Elf Warrior,Winnowing Forces's power and toughness are eac...
317,Vertex Paladin,*,Creature — Angel Knight,Flying\nVertex Paladin's power and toughness a...


In [55]:
# Parsing the toughness column
set_data = (set_data
        .apply(
            parse_numeric_col,
            axis='columns',
            col_name='toughness',
            value_regex=NUMBER_REGEX))

In [56]:
# Casting the toughness column as float16
set_data.static_toughness = set_data.static_toughness.astype('float16')

In [57]:
# Displaing an overview of the data parsed from the static values of 
# the toughness column
set_data.loc[
    set_data.toughness_type == 'static',
    ['name','toughness','toughness_type','static_toughness']]

,name,toughness,toughness_type,static_toughness
3,Accomplished Alchemist,5.0,static,5.0
5,Aetherblade Agent,1.0,static,1.0
6,Gitaxian Mindstinger,3.0,static,3.0
8,Ageless Guardian,4.0,static,4.0
9,Akki Scrapchomper,1.0,static,1.0
...,...,...,...,...
642,Zephyr Winder,1.0,static,1.0
643,Zhalfirin Lancer,3.0,static,3.0
645,Zimone and Dina,4.0,static,4.0
646,"Zimone, Quandrix Prodigy",2.0,static,2.0


In [58]:
# Displaing an overview of the data parsed from the dynamic values of 
# the toughness column
set_data.loc[
    set_data.toughness_type == 'dynamic',
    ['name','toughness','toughness_type','static_toughness']]


,name,toughness,toughness_type,static_toughness
285,Winnowing Forces,*,dynamic,0.0
317,Vertex Paladin,*,dynamic,0.0


In [59]:
# Count the number of appearances of al non-null values in the toughness
# column
set_data.toughness.value_counts()


2.0     90
3.0     74
4.0     72
1.0     48
5.0     31
6.0     10
7.0      3
0.0      3
8.0      2
*        2
10.0     1
11.0     1
12.0     1
Name: toughness, dtype: int64

In [60]:
# Display an array with all the unique values in the toughness column
set_data.toughness.unique()

array([nan, '5.0', '1.0', '3.0', '4.0', '2.0', '6.0', '7.0', '0.0', '8.0',
       '10.0', '11.0', '12.0', '*'], dtype=object)

Lastly, we can make sure that all the values in the power and toughness do correlate to creatures and artifacts.

In [64]:
(set_data
    .loc[
        :,
        ['name','power','toughness', 'is_creature','is_artifact']]
    .groupby(
        ['is_creature','is_artifact'])
    .count()
    )

name  power  toughness
is_creature is_artifact                        
False       False         291      0          0
            True           22      3          3
True        False         324    324        324
            True           11     11         11

In [63]:
set_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cmc               648 non-null    int64  
 1   colors            648 non-null    object 
 2   defense           36 non-null     float64
 3   face_type         648 non-null    object 
 4   id                648 non-null    object 
 5   is_artifact       648 non-null    bool   
 6   is_battle         648 non-null    bool   
 7   is_black          648 non-null    bool   
 8   is_blue           648 non-null    bool   
 9   is_colorless      648 non-null    bool   
 10  is_creature       648 non-null    bool   
 11  is_enchantment    648 non-null    bool   
 12  is_green          648 non-null    bool   
 13  is_instant        648 non-null    bool   
 14  is_land           648 non-null    bool   
 15  is_legendary      648 non-null    bool   
 16  is_planeswalker   648 non-null    bool   
 1

In [61]:
set_data.head()

,cmc,colors,defense,face_type,id,is_artifact,is_battle,is_black,is_blue,is_colorless,...,normal_image_uri,oracle_text,power,power_type,rarity,static_power,static_toughness,toughness,toughness_type,type_line
0,1,R,NaN,front_face,4620cc3b-e401-4096-b310-fed080806344,False,False,False,False,False,...,https://cards.scryfall.io/normal/front/4/6/462...,Target creature blocks this turn if able. You ...,NaN,NaN,uncommon,NaN,NaN,NaN,NaN,Instant
1,2,W,NaN,front_face,05521edf-f47f-4e7a-aec5-cdc4ae7368c2,False,False,False,False,False,...,https://cards.scryfall.io/normal/front/0/5/055...,Choose one —\n• Choose a nonland card name. Op...,NaN,NaN,rare,NaN,NaN,NaN,NaN,Sorcery — Lesson
2,0,colorless,NaN,front_face,edf8eb51-9643-4c54-b38e-e7abea92bbe1,False,False,False,False,True,...,https://cards.scryfall.io/normal/front/e/d/edf...,"{T}: Add {C}.\n{3}, {T}: Target creature with ...",NaN,NaN,uncommon,NaN,NaN,NaN,NaN,Land
3,4,G,NaN,front_face,0d7b7830-b65e-4c53-98e8-152026764e4b,False,False,False,False,False,...,https://cards.scryfall.io/normal/front/0/d/0d7...,{T}: Add one mana of any color.\n{T}: Add X ma...,2.0,static,rare,2.0,5.0,5.0,static,Creature — Elf Druid
4,2,W,NaN,front_face,f7017afb-4c7c-4c8d-9c9d-3f056a55561e,False,False,False,False,False,...,https://cards.scryfall.io/normal/front/f/7/f70...,Convoke (Your creatures can help cast this spe...,NaN,NaN,common,NaN,NaN,NaN,NaN,Instant
